In [158]:
import logging

def get_rotpos(steps, turns):
    print(f"get_rotpos: {steps/spt=}, {turns=}")
    return steps/spt + turns
    
def steps_turn_to_az(steps, turns):
    rotpos = get_rotpos(steps, turns)
    print(f"steps_turn_to_az: {rotpos=}, {rotpos-north_rotpos=}")
    return ((rotpos - north_rotpos)*degree_per_turn) % 360
    

# 0 azimuth is North
# 360 = 0
spt = 2**14
tpr = 5
degree_per_turn = 360/tpr
degree_per_step = degree_per_turn/spt
turn_per_degree = tpr/360
home_az = 90
home_steps = 10000
home_turns = 2
home_rotpos = get_rotpos(home_steps, home_turns)
north_rotpos = home_rotpos + ((0 - home_az)/degree_per_turn)
assert degree_per_step*spt == degree_per_turn
print(f"{degree_per_step=}\n{degree_per_turn=}\n{turn_per_degree=}\n{home_rotpos=}\n{north_rotpos=}")

get_rotpos: steps/spt=0.6103515625, turns=2
degree_per_step=0.00439453125
degree_per_turn=72.0
turn_per_degree=0.013888888888888888
home_rotpos=2.6103515625
north_rotpos=1.3603515625


In [159]:
print("should be 90:", steps_turn_to_az(10000, 2))

get_rotpos: steps/spt=0.6103515625, turns=2
steps_turn_to_az: rotpos=2.6103515625, rotpos-north_rotpos=1.25
should be 90: 90.0


In [160]:
steps_turn_to_az(10000, 7)

get_rotpos: steps/spt=0.6103515625, turns=7
steps_turn_to_az: rotpos=7.6103515625, rotpos-north_rotpos=6.25


90.0

In [163]:
RIGHT = 1
LEFT = 2
limits = {LEFT: -270, RIGHT:270}

def direction_finder(current_az, target_az):
    diff = abs(target_az - current_az)
    is_left = (target_az - current_az) < 0
    return is_left, diff
    #back_dir = abs(current_az + (360-target_az))
    #return forward_dir, back_dir

def new_direction_finder(current_az, target_az, limits, limitscounter):
    diff1 = (target_az - current_az)%360
    if diff1 == 0:
        direction = RIGHT
        diff = 0
    if diff1 == 180:
        print("180 degrees, going towards the shortest cable")
        direction = RIGHT if limitscounter[RIGHT] < abs(limitscounter[LEFT]) else LEFT
    if current_az + 180 > target_az:
        # Rotate current directly towards target.
        direction = RIGHT
        diff = diff1
    else:
        # Rotate the other direction towards target.
        direction = LEFT
        diff = current_az + 360 - target_az

    print(f"Orig result: {'LEFT' if direction==LEFT else 'RIGHT'}, {diff=}")
    if direction == LEFT:
        total_left = limitscounter[LEFT] - diff
        print(f"{total_left=}, {limitscounter[RIGHT]=}")
        if total_left+limitscounter[RIGHT] < limits[LEFT]:
            print("cable length violation")
            direction = RIGHT
            diff = 360 - diff
    else:
        total_right = limitscounter[RIGHT] + diff
        print(f"{limitscounter[LEFT]=}, {total_right=}")
        if limitscounter[LEFT]+total_right > limits[RIGHT]:
            print("cable length violation")
            direction = LEFT
            diff = 360 - diff
        
    return direction, diff

import math

def shortest_path(currentaz, targetaz):
    diff_rad = math.radians(currentaz) - math.radians(targetaz)
    diff_rad2 = math.radians(currentaz) - math.radians(targetaz)
    return diff_rad, diff_rad2, math.degrees(abs(diff_rad))
    
print(shortest_path(90,350))
print(math.radians(90), math.radians(350))

(-4.537856055185257, -4.537856055185257, 260.0)
1.5707963267948966 6.1086523819801535


In [164]:

t1=(90,100)
r1=(RIGHT, 10)
t2=(90,350)
r2=(LEFT, 100)
t3=(350,10)
r3=(RIGHT, 20)
t4=(359,0)
r4=(RIGHT, 1)
t5=(0,180)
r5=(RIGHT, 180)
t6=(10,359)
r6=(LEFT, 11)


tests = [t1, t2, t3, t4, t5, t6]
results = [r1, r2, r3, r4, r5, r6]

print(limits)
print(limitscounter)
for idx, test in enumerate(tests):
    limitscounter = {LEFT: -260, RIGHT: 0}
    direction, diff = new_direction_finder(*test, limits, limitscounter)
    
    print(f"TEST: {test=}, {'LEFT' if direction == LEFT else 'RIGHT'}, {diff=}, {direction == results[idx][0] and diff == results[idx][1]}")
    print(f"Expected: {'LEFT' if results[idx][0] == LEFT else 'RIGHT'}, {results[idx][1]}\n")
    



{2: -270, 1: 270}
{2: -260, 1: 0}
Orig result: RIGHT, diff=10
limitscounter[LEFT]=-260, total_right=10
TEST: test=(90, 100), RIGHT, diff=10, True
Expected: RIGHT, 10

Orig result: LEFT, diff=100
total_left=-360, limitscounter[RIGHT]=0
cable length violation
TEST: test=(90, 350), RIGHT, diff=260, False
Expected: LEFT, 100

Orig result: RIGHT, diff=20
limitscounter[LEFT]=-260, total_right=20
TEST: test=(350, 10), RIGHT, diff=20, True
Expected: RIGHT, 20

Orig result: RIGHT, diff=1
limitscounter[LEFT]=-260, total_right=1
TEST: test=(359, 0), RIGHT, diff=1, True
Expected: RIGHT, 1

180 degrees, going towards the shortest cable
Orig result: LEFT, diff=180
total_left=-440, limitscounter[RIGHT]=0
cable length violation
TEST: test=(0, 180), RIGHT, diff=180, True
Expected: RIGHT, 180

Orig result: LEFT, diff=11
total_left=-271, limitscounter[RIGHT]=0
cable length violation
TEST: test=(10, 359), RIGHT, diff=349, False
Expected: LEFT, 11



In [62]:
isinstance('N', str)

True

In [ ]:
cables = {"LEFT": 0, "LEFT_MAX": -270, "RIGHT": 0, "RIGHT_MAX": 270}




In [120]:
700%360

340

In [49]:
from dataclasses import dataclass

@dataclass(init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False)
class DomePos:
    # don't change externally!
    steps: int
    turns: int
    az: float
    rotpos: float = -1

    def __init__(self, steps, turns, az):
        self.steps = steps
        self.turns = turns
        self.az = az

    @property
    def alt(self) -> int:
        return 0


In [50]:
a = DomePos(steps=0, turns=1, az=90)

In [51]:
a

DomePos(steps=0, turns=1, az=90, rotpos=-1)